In [2]:
import gym
import numpy as np
import os
import argparse
import time

import pandas as pd
import plotly
import plotly.express as px
from plotly.subplots import make_subplots

import json
# import matplotlib.pyplot as plt

from stable_baselines3 import DDPG, PPO
from smart_nanogrid_gym.utils.config import solvers_files_directory_path

In [459]:
json_path_initial_values = r'solvers/RL/training_files/PPO-b-pv-controlled-dense-8ch-1h-initial_values.json'
json_path_predictions = r'solvers/RL/training_files/PPO-b-pv-controlled-dense-8ch-1h-prediction_results.json'

In [85]:
pd.set_option('display.max_columns', None)

In [460]:
with open(json_path_predictions) as json_data:
    predict_data = json.load(json_data)
    df_predict_soc = pd.DataFrame(predict_data['SOC'])
    df_predict_grid_p = pd.DataFrame(predict_data['Grid_power'])
    df_predict_grid_e = pd.DataFrame(predict_data['Grid_energy'])
    df_predict_util_sol = pd.DataFrame(predict_data['Utilized_solar_energy'])
    df_predict_avail_sol = pd.DataFrame(predict_data['Available_solar_energy'])
    df_predict_ev_penalties = pd.DataFrame(predict_data['Vehicle_penalties'])
    df_predict_bess_penalties = pd.DataFrame(predict_data['Battery_penalties'])
    df_predict_penalties = pd.DataFrame(predict_data['Total_penalties'])
    df_predict_total_cost = pd.DataFrame(predict_data['Total_cost'])
    df_predict_battery_soc = pd.DataFrame(predict_data['Battery_state_of_charge'])
    df_predict_grid_e_cost = pd.DataFrame(predict_data['Grid_energy_cost'])
    df_predict_bess_action = pd.DataFrame(predict_data['Battery_action'])
    df_predict_charger_actions = pd.DataFrame(predict_data['Charger_actions'])        
    df_predict_total_ch_p = pd.DataFrame(predict_data['Total_charging_power'])
    df_predict_total_disch_p = pd.DataFrame(predict_data['Total_discharging_power'])
    df_predict_ch_p_vals = pd.DataFrame(predict_data['Charger_power_values'])
    df_predict_bess_p_val = pd.DataFrame(predict_data['Battery_power_value'])    

In [461]:
with open(json_path_initial_values) as json_data:
    init_data = json.load(json_data)
    df_init_soc = pd.DataFrame(init_data['SOC'])
    df_init_arr = pd.DataFrame(init_data['Arrivals'])    
    df_init_dep = pd.DataFrame(init_data['Departures'])    
    df_init_co = pd.DataFrame(init_data['Charger_occupancy'])
    df_init_vcap = pd.DataFrame(init_data['Vehicle_capacities'])
    df_init_req_soc = pd.DataFrame(init_data['Requested_SOC'])    

In [88]:
pd.options.plotting.backend = "plotly"

# Initial values

In [467]:
df_init_soc

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,0.690107,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.486255,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.876665,0.0,0.0,0.0,0.000000,0.0
1,0.835481,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.708192,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
2,0.000000,0.0,0.000000,0.563477,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.272853,0.0,0.000000,0.000000,0.000000,0.358535,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
3,0.524369,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.33034,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
4,0.000000,0.0,0.000000,0.000000,0.0,0.877816,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.499986,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
5,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.652768,0.0,0.0,0.000000,0.00000,0.000000,0.648206,0.0,0.000000,0.000000,0.000000,0.000000,0.173478,0.000000,0.0,0.0,0.0,0.116196,0.0
6,0.644472,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.748935,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
7,0.000000,0.0,0.295091,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.233941,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.204461,0.0,0.0,0.0,0.000000,0.0


In [468]:
df_init_soc = df_init_soc.T

df_initial_soc = pd.DataFrame()

for charger in df_init_soc.columns:
    temp_df = pd.DataFrame(df_init_soc[charger])
    temp_df.columns = ['State of Charge']
    temp_df['Charger ID'] = str(charger)
    df_initial_soc = pd.concat([df_initial_soc, temp_df])

df_initial_soc

,State of Charge,Charger ID
0,0.690107,0
1,0.000000,0
2,0.000000,0
3,0.000000,0
4,0.000000,0
...,...,...
20,0.000000,7
21,0.000000,7
22,0.000000,7
23,0.000000,7


In [469]:
fig = df_initial_soc.plot(kind='bar', title="Initial state of charge on arrival", template="plotly_white", color="Charger ID",
                          y="State of Charge", facet_row="Charger ID", facet_row_spacing=0.05,
                          labels=dict(index="Timestep (hour) in a Day", value="State of Charge", variable="Charger ID"))
fig.update_xaxes(dtick="M1", tickformat="%b", matches='x', griddash="dot")
fig.update_yaxes(tickformat="%b", matches='y', title='', tickvals=[i/10 for i in range(0, 11, 2)])


fig.for_each_annotation(lambda a: a.update(text="", font_size=24, y=1.07))
fig.add_annotation(x=-0.1, y=0.5, text="State of Charge", textangle=-90, xref="paper", yref="paper", 
                   font_size=14, showarrow=False)
fig.update_layout(height=550, width=750, title=dict(xanchor="center", x=0.5, font_size=24),
                  legend=dict(orientation="h", yanchor="top", y=1.08, xanchor="center", x=0.5))
fig.show()

In [92]:
df_arrivals = df_init_arr.copy()
df_departures = df_init_dep.copy()
df_arrivals['ID'] = 'Arrivals'
df_departures['ID'] = 'Departures'
df_vehicle_shifts = pd.concat([df_arrivals, df_departures])
df_vehicle_shifts['Charger ID'] = df_vehicle_shifts.index
df_vehicle_shifts

,0,1,2,3,ID,Charger ID
0,0,13,NaN,NaN,Arrivals,0
1,0,5,23.0,NaN,Arrivals,1
2,2,11,19.0,NaN,Arrivals,2
3,1,9,16.0,NaN,Arrivals,3
4,1,6,14.0,23.0,Arrivals,4
5,3,12,22.0,NaN,Arrivals,5
6,1,11,21.0,NaN,Arrivals,6
7,0,10,17.0,NaN,Arrivals,7
0,8,21,NaN,NaN,Departures,0
1,4,9,27.0,NaN,Departures,1


In [93]:
fig = df_vehicle_shifts.plot(kind="scatter", title="", template="plotly_white", x='Charger ID',
                             y=[0, 1, 2, 3], facet_col='ID', facet_col_spacing=0.2,
                             labels=dict(index="Charger ID", value="Timestep (Hour) in a Day", variable="Vehicle ID per Charger"))
fig.update_yaxes(dtick="M1", tickformat="%b", griddash="dot", tickvals=[i for i in range(0, 25, 1)])
fig.update_xaxes(dtick="M1", tickformat="%b")
fig.update_traces(marker_size=10)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1], font_size=24, y=1.07))
fig.update_layout(width=700, title=dict(xanchor="center", x=0.5, font_size=24),
                  legend=dict(orientation="h", yanchor="top", y=1.06, xanchor="center", x=0.5))
fig.show()

In [94]:
df_init_co = df_init_co.T

df_charger_occupancy = pd.DataFrame()

for charger in df_init_co.columns:
    temp_df = pd.DataFrame(df_init_co[charger])
    temp_df.columns = ['State']
    temp_df['Charger ID'] = charger
    df_charger_occupancy = pd.concat([df_charger_occupancy, temp_df])

df_charger_occupancy

,State,Charger ID
0,1.0,0
1,1.0,0
2,1.0,0
3,1.0,0
4,1.0,0
...,...,...
20,1.0,7
21,1.0,7
22,1.0,7
23,0.0,7


In [95]:
fig = df_charger_occupancy.plot(kind="line", title="Occupancy per Charger", template="plotly_white", y='State', 
                                facet_row='Charger ID', facet_row_spacing=0.05, color='Charger ID', 
                                labels=dict(index="Timestep (Hour) in a Day", value="State", variable=""))
fig.update_yaxes(dtick="M1", tickformat="%b", ticktext=["Unoccupied", "Occupied"], 
                 tickvals=[0, 1], matches='y', title='')
fig.update_xaxes(dtick="M1", tickformat="%b", matches='x', griddash="dot")
fig.update_traces(marker_size=10)
fig.for_each_annotation(lambda a: a.update(text="", font_size=24, y=1.07))
fig.add_annotation(x=-0.2, y=0.5, text="State", textangle=-90, xref="paper", yref="paper", font_size=14, showarrow=False)
fig.update_layout(height=550, width=750, title=dict(xanchor="center", x=0.5, font_size=24),
                  legend=dict(orientation="h", yanchor="top", y=1.08, xanchor="center", x=0.5))
fig.show()

In [96]:
df_init_req_soc = df_init_req_soc.T

df_requested_soc = pd.DataFrame()

for charger in df_init_req_soc.columns:
    temp_df = pd.DataFrame(df_init_req_soc[charger])
    temp_df.columns = ['Requested State of Charge']
    temp_df['Charger ID'] = charger
    df_requested_soc = pd.concat([df_requested_soc, temp_df])

df_requested_soc

,Requested State of Charge,Charger ID
0,1.0,0
1,1.0,0
2,1.0,0
3,1.0,0
4,1.0,0
...,...,...
20,1.0,7
21,1.0,7
22,1.0,7
23,0.0,7


In [97]:
fig = df_requested_soc.plot(kind="line", title="Requested State of Charge per Vehicle", template="plotly_white", 
                            y='Requested State of Charge', facet_row='Charger ID', facet_row_spacing=0.05, color='Charger ID', 
                            labels=dict(index="Timestep (Hour) in a Day", value="", variable=""))
fig.update_yaxes(tickformat="%b", matches='y', title='')
fig.update_xaxes(dtick="M1", tickformat="%b", matches='x', griddash="dot")
fig.update_traces(marker_size=10)
fig.for_each_annotation(lambda a: a.update(text="", font_size=24, y=1.07))
fig.add_annotation(x=-0.08, y=0.5, text="Requested State of Charge", textangle=-90, 
                   xref="paper", yref="paper", font_size=14, showarrow=False)
fig.update_layout(height=550, width=750, title=dict(xanchor="center", x=0.5, font_size=24),
                  legend=dict(orientation="h", yanchor="top", y=1.08, xanchor="center", x=0.5))
fig.show()

In [462]:
df_init_vcap = df_init_vcap.T

df_vehicle_capacity = pd.DataFrame()

for charger in df_init_vcap.columns:
    temp_df = pd.DataFrame(df_init_vcap[charger])
    temp_df.columns = ['Vehicle Capacities']
    temp_df['Charger ID'] = charger
    df_vehicle_capacity = pd.concat([df_vehicle_capacity, temp_df])

df_vehicle_capacity

,Vehicle Capacities,Charger ID
0,81.0,0
1,81.0,0
2,81.0,0
3,81.0,0
4,81.0,0
...,...,...
20,15.0,7
21,15.0,7
22,15.0,7
23,0.0,7


In [463]:
fig = df_vehicle_capacity.plot(kind="line", line_shape="hvh", title="Battery Capacity per Vehicle", template="plotly_white", 
                               y='Vehicle Capacities', facet_row='Charger ID', facet_row_spacing=0.05, color='Charger ID', 
                               markers=True, labels=dict(index="Timestep (Hour) in a Day", value="", variable=""))
fig.update_yaxes(tickformat="%b", matches='y', title='')
fig.update_xaxes(dtick="M1", tickformat="%b", matches='x', griddash="dot", tickvals=[i for i in range(0, 25, 1)])
fig.update_traces(marker_size=7)
fig.for_each_annotation(lambda a: a.update(text="", font_size=24, y=1.07))
fig.add_annotation(x=-0.1, y=0.5, text="Vehicle Capacity (kWh)", textangle=-90, 
                   xref="paper", yref="paper", font_size=14, showarrow=False)
fig.update_layout(height=550, width=850, title=dict(xanchor="center", x=0.5, font_size=24),
                  legend=dict(orientation="h", yanchor="top", y=1.08, xanchor="center", x=0.5))
fig.show()

# Predictions

In [464]:
df_predict_soc

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,0.948131,0.997407,0.999870,0.999994,1.000000,1.000000,0.000000,0.000000,0.000000,0.784827,0.989241,0.999462,0.999973,0.999999,1.000000,1.000000,1.000000,1.000000,0.000000,0.993833,0.999692,0.999985,0.999999,0.000000,0.0
1,0.991774,0.999589,0.999979,0.999999,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.985410,0.999270,0.999964,0.999998,1.000000,1.000000,1.000000,1.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.748808,0.987440,0.999372,0.999969,0.999998,1.000000,1.000000,0.000000,0.000000,0.853409,0.992670,0.999634,0.999982,0.000000,0.967927,0.998396,0.999920,0.999996,0.999999,1.000000,0.000000,0.0
3,0.976218,0.998148,0.999267,0.999963,0.999994,1.000000,1.000000,1.000000,0.000000,0.000000,0.945046,0.997144,0.998943,0.999947,0.999997,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.951594,0.982433,0.993143,0.999657,0.999918,0.999996,0.999999,0.000000,0.000000,0.000000,0.000000,0.974999,0.998750,0.999628,0.999924,0.999982,0.999993,0.999999,1.000000,0.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.890268,0.994513,0.999726,0.999986,0.999999,0.000000,0.982410,0.999121,0.999956,0.999998,0.000000,0.000000,0.490442,0.844679,0.992234,0.999204,0.000000,0.542727,0.0
6,0.982224,0.999111,0.999956,0.999998,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.987447,0.999372,0.999969,0.999998,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.0
7,0.000000,0.000000,0.675091,0.983755,0.999188,0.999959,0.000000,0.000000,0.000000,0.000000,0.000000,0.708941,0.985447,0.999272,0.999964,0.000000,0.000000,0.000000,0.000000,0.960223,0.993240,0.999662,0.999983,0.000000,0.0


In [465]:
df_predict_soc = df_predict_soc.T

df_predicted_vehicle_soc = pd.DataFrame()

for charger in df_predict_soc.columns:
    temp_df = pd.DataFrame(df_predict_soc[charger])
    temp_df.columns = ['Vehicle State of Charge']
    temp_df['Charger ID'] = charger
    df_predicted_vehicle_soc = pd.concat([df_predicted_vehicle_soc, temp_df])

df_predicted_vehicle_soc

,Vehicle State of Charge,Charger ID
0,0.948131,0
1,0.997407,0
2,0.999870,0
3,0.999994,0
4,1.000000,0
...,...,...
20,0.993240,7
21,0.999662,7
22,0.999983,7
23,0.000000,7


In [466]:
fig = df_predicted_vehicle_soc.plot(kind="area", title="Predicted State of Charge per Vehicle", 
                                    template="plotly_white", y='Vehicle State of Charge', facet_row='Charger ID', 
                                    facet_row_spacing=0.05, color='Charger ID', markers=True, 
                                    labels=dict(index="Timestep (Hour) in a Day", value="", variable=""))
fig.update_yaxes(tickformat="%b", matches='y', title='')
fig.update_xaxes(dtick="M1", tickformat="%b", matches='x', griddash="dot", tickvals=[i for i in range(0, 25, 1)])
fig.update_traces(marker_size=7)
fig.for_each_annotation(lambda a: a.update(text="", font_size=24, y=1.07))
fig.add_annotation(x=-0.08, y=0.5, text="Vehicle State of Charge", textangle=-90, 
                   xref="paper", yref="paper", font_size=14, showarrow=False)
fig.update_layout(height=800, width=850, title=dict(xanchor="center", x=0.5, font_size=24),
                  legend=dict(orientation="h", yanchor="top", y=1.05, xanchor="center", x=0.5))
fig.show()

In [383]:
df_predict_avail_sol

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.693348,8.071687,12.211495,13.914825,13.8459,12.565621,11.239753,11.119133,10.17697,7.632897,2.402762,0.165164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.008407,4.434642,2.849014,1.188423,4.739312,8.666373,11.838578,13.321631,12.161972,9.417228,3.818585,0.324547,0.0,0.0,0.0,0.0,0.0


In [384]:
fig = df_predict_avail_sol.T.plot(title="Predicted available solar energy", line_shape= "linear", template="plotly_white", 
                       labels=dict(index="Timestep (Hour) in a Day (For 2 Days)", value="Solar Energy (kWh)", variable="Charger ID"))
fig.update_yaxes(dtick="M1", tickformat="%b")
fig.update_xaxes(dtick="M1", tickformat="%b", griddash="dot", tickvals=[i for i in range(0, 48, 2)])
fig.update_traces(marker_size=10)
fig.update_layout(width=980, title=dict(xanchor="center", x=0.5, font_size=24, y=0.9), showlegend=False)
fig.show()

In [385]:
df_predict_util_sol.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.693348,8.071687,12.211495,13.914825,13.8459,12.565621,11.239753,11.119133,10.17697,7.632897,2.402762,0.165164,0.0,0.0,0.0,0.0,0.0


In [386]:
fig = df_predict_util_sol.plot(title="Predicted utilised solar energy", line_shape= "linear", template="plotly_white", 
                       labels=dict(index="Timestep (Hour) in a Day", value="Solar Energy (kWh)"))
fig.update_yaxes(dtick="M1", tickformat="%b")
fig.update_xaxes(dtick="M1", tickformat="%b", griddash="dot")
fig.update_traces(marker_size=10)
fig.update_layout(width=950, title=dict(xanchor="center", x=0.5, font_size=24, y=0.9), showlegend=False)
fig.show()

In [470]:
df_predict_battery_soc.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,-18442.175077,-18442.175077,-18442.175077,-18442.175077,-18442.175077,-18442.175077,-18442.175077,-18442.175077,-18442.509109,-18442.509109,-18442.509109,-18442.509109,-18442.509109,-18442.811165,-18442.811165,-18442.802415,-18442.802415,-18442.802415,-18442.802415,-18442.802415,-18442.802415,-18442.802415,-18442.802415,-18442.802415


In [471]:
fig = df_predict_battery_soc.plot(title="Predicted Battery State of Charge", line_shape= "linear", template="plotly_white", 
                       labels=dict(index="Timestep (Hour) in a Day", value="State of Charge"))
fig.update_yaxes(tickformat="%b")
fig.update_xaxes(dtick="M1", tickformat="%b", griddash="dot")
fig.update_traces(marker_size=10)
fig.update_layout(width=950, title=dict(xanchor="center", x=0.5, font_size=24, y=0.9), showlegend=False)
fig.show()

In [472]:
df_predict_grid_e.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,57.308525,5.783206,21.19015,31.845062,19.94131,9.334204,24.432513,7.692847,0.0,8.740957,21.304109,9.541291,41.31821,0.0,8.535034,0.0,10.729266,18.625237,19.57535,39.562422,9.568915,0.526267,0.005976,20.900024


In [473]:
fig = df_predict_grid_e.plot(title="Predicted Grid Energy", line_shape= "linear", template="plotly_white", 
                       labels=dict(index="Timestep (Hour) in a Day", value="Grid Energy (kWh)"))
fig.update_yaxes(tickformat="%b")
fig.update_xaxes(dtick="M1", tickformat="%b", griddash="dot")
fig.update_traces(marker_size=10)
fig.update_layout(width=950, title=dict(xanchor="center", x=0.5, font_size=24, y=0.9), showlegend=False)
fig.show()

In [474]:
df_predict_grid_p.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,57.308525,5.783206,21.19015,31.845062,19.94131,9.334204,24.432513,7.692847,0.0,8.740957,21.304109,9.541291,41.31821,0.0,8.535034,0.0,10.729266,18.625237,19.57535,39.562422,9.568915,0.526267,0.005976,20.900024


In [475]:
fig = df_predict_grid_p.plot(title="Predicted Grid Power", line_shape= "linear", template="plotly_white", 
                       labels=dict(index="Timestep (Hour) in a Day", value="Grid Power (kW)"))
fig.update_yaxes(tickformat="%b")
fig.update_xaxes(dtick="M1", tickformat="%b", griddash="dot")
fig.update_traces(marker_size=10)
fig.update_layout(width=950, title=dict(xanchor="center", x=0.5, font_size=24, y=0.9), showlegend=False)
fig.show()

In [476]:
df_predict_grid_e_cost.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,6.587424,0.66476,2.435737,3.660484,2.292187,1.072936,2.808436,1.468821,0.0,1.66894,4.067665,1.82175,7.889024,0.0,1.629622,0.0,2.048574,3.556179,3.737587,7.553785,1.099915,0.060493,0.000687,2.402388


In [477]:
fig = df_predict_grid_e_cost.plot(kind="bar", title="Predicted Grid Energy Cost", template="plotly_white", 
                       labels=dict(index="Timestep (Hour) in a Day", value="Energy Cost (€)"))
fig.update_yaxes(tickformat="%b")
fig.update_xaxes(dtick="M1", tickformat="%b", griddash="dot")
fig.update_layout(width=950, title=dict(xanchor="center", x=0.5, font_size=24, y=0.9), showlegend=False)
fig.show()

In [478]:
df_predict_penalties.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0.99406,1.004822,0.99406,1.416324,1.24645,0.99406,0.99406,1.042225,0.639294,0.639294,0.824492,0.651374,0.978154,0.664054,0.578098,0.0,0.0,0.022888,0.0,1.038598,0.096498,0.0,0.0,0.0


In [479]:
fig = df_predict_penalties.plot(kind="line", title="Agent penalties", template="plotly_white", 
                       labels=dict(index="Timestep (Hour) in a Day", value="Penalty"))
fig.update_yaxes(tickformat="%b")
fig.update_xaxes(dtick="M1", tickformat="%b", griddash="dot")
fig.update_layout(width=950, title=dict(xanchor="center", x=0.5, font_size=24, y=0.9), showlegend=False)
fig.show()

In [480]:
df_predict_total_cost.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,7.581484,1.669582,3.429797,5.076808,3.538637,2.066996,3.802496,2.511046,0.639294,2.308234,4.892157,2.473124,8.867178,0.664054,2.20772,0.0,2.048574,3.579067,3.737587,8.592383,1.196413,0.060493,0.000687,2.402388


In [481]:
df_predict_total_cost = df_predict_total_cost * (-1)

In [482]:
fig = df_predict_total_cost.plot(kind="line", title="Reward - Total Cost", template="plotly_white", 
                       labels=dict(index="Timestep (Hour) in a Day", value="Reward"))
fig.update_yaxes(tickformat="%b")
fig.update_xaxes(dtick="M1", tickformat="%b", griddash="dot")
fig.update_layout(width=950, title=dict(xanchor="center", x=0.5, font_size=24, y=0.9), showlegend=False)
fig.show()

In [483]:
df_predict_total_ch_p.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,57.308525,5.783206,21.19015,31.845062,19.94131,9.334204,24.432513,10.386195,1.194949,20.952453,35.218935,23.38719,53.883831,6.693693,19.654167,6.248326,18.362163,21.027999,19.740514,39.562422,9.568915,0.526267,0.005976,20.900024


In [484]:
df_predict_total_disch_p.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [485]:
df_predict_charger_actions.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.000000,1.000000,1.000000,1.000000,1.000000,0.788449,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.676282,1.000000,1.000000,1.000000
2,1.000000,1.000000,1.000000,0.709403,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.842256,1.000000,1.000000
3,1.000000,0.970638,0.636128,1.000000,0.871079,1.000000,1.000000,1.000000,1.0,0.532308,1.0,0.997929,0.662982,1.000000,1.000000,1.000000,0.962558,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.968122
4,0.967234,0.927050,1.000000,1.000000,0.771210,0.635607,0.670613,0.641735,1.0,0.801810,1.0,0.895415,1.000000,0.969664,0.595539,0.702086,1.000000,1.000000,0.739476,0.838335,0.801153,0.665612,0.944955,0.989658
5,0.807068,1.000000,1.000000,1.000000,1.000000,0.822207,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.894777,1.000000,1.000000,0.944755,1.000000,1.000000
6,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.644816,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.990718,1.000000,1.000000,0.998610,1.000000,1.000000,1.000000
7,1.000000,0.953284,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.870239,1.000000,1.000000,1.000000,1.000000,0.873747,1.000000,1.000000,1.000000


In [487]:
df_predict_charger_actions = df_predict_charger_actions.T

df_predicted_ch_actions = pd.DataFrame()

for charger in df_predict_charger_actions.columns:
    temp_df = pd.DataFrame(df_predict_charger_actions[charger])
    temp_df.columns = ['Action Value']
    temp_df['Charger ID'] = str(charger)
    df_predicted_ch_actions = pd.concat([df_predicted_ch_actions, temp_df])

df_predicted_ch_actions

,Action Value,Charger ID
0,1.000000,0
1,1.000000,0
2,1.000000,0
3,1.000000,0
4,1.000000,0
...,...,...
19,1.000000,7
20,0.873747,7
21,1.000000,7
22,1.000000,7


In [488]:
fig = df_predicted_ch_actions.plot(kind="bar", title="Actions per charger", 
                                    template="plotly_white", y='Action Value', facet_row='Charger ID', 
                                    facet_row_spacing=0.05, color='Charger ID', 
                                    labels=dict(index="Timestep (Hour) in a Day", value="", variable=""))
fig.update_yaxes(tickformat="%b", matches='y', title='')
fig.update_xaxes(dtick="M1", tickformat="%b", matches='x', griddash="dot", tickvals=[i for i in range(0, 25, 1)])

fig.for_each_annotation(lambda a: a.update(text="", font_size=24, y=1.07))
fig.add_annotation(x=-0.08, y=0.5, text="Charger action value", textangle=-90, 
                   xref="paper", yref="paper", font_size=14, showarrow=False)
fig.update_layout(height=800, width=850, title=dict(xanchor="center", x=0.5, font_size=24, y=0.95),
                  legend=dict(orientation="h", yanchor="top", y=1.05, xanchor="center", x=0.5))
fig.show()

In [489]:
df_predict_ch_p_vals.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,20.900000,3.991304,0.199565,0.009978,0.000499,0.000025,0.000000e+00,0.000000e+00,0.000000e+00,20.900000,14.309017,0.715451,0.035773,0.001789,0.000089,0.000004,2.235784e-07,1.117892e-08,0.000000,7.264417,3.632208e-01,1.816104e-02,9.080521e-04,0.000000
1,5.157673,0.257884,0.012894,0.000645,0.000032,0.000001,4.045125e-07,2.022563e-08,1.011281e-09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.267138,2.633569e-01,1.316785e-02,0.000658,0.000033,1.113148e-06,5.884906e-07,2.942453e-08,0.000000
2,0.000000,0.000000,0.000000,14.826525,19.090576,0.954529,4.772644e-02,2.386322e-03,1.193161e-04,0.000006,0.000000,0.000000,20.900000,5.013414,0.250671,0.012534,0.000000e+00,2.010991e+01,1.005496,0.050275,2.513739e-03,1.058605e-04,2.511944e-05,0.000000
3,19.429538,0.942953,0.048137,0.029942,0.001304,0.000258,1.291048e-05,6.455239e-07,0.000000e+00,0.000000,20.900000,1.771347,0.061155,0.034145,0.001707,0.000085,4.108354e-06,3.652259e-07,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,8.336946,3.484774e+00,1.210227e+00,7.361511e-01,0.029513,0.008771,0.000393,0.000000,0.000000,0.000000,0.000000,1.805050e+01,9.025251e-01,0.033370,0.011255,2.189616e-03,4.346087e-04,2.268528e-04,0.000024
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.090000e+01,9.173581e+00,4.586790e-01,0.022934,0.001147,0.000000,20.720656,1.036033,0.051802,0.002590,0.000000e+00,0.000000e+00,18.700848,20.900000,8.705731e+00,4.112392e-01,0.000000e+00,20.900000
6,11.821315,0.591066,0.029553,0.001478,0.000074,0.000004,1.191027e-07,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,19.319482,0.965974,4.829871e-02,2.392519e-03,0.000142,0.000007,3.546116e-07,1.822419e-08,9.112056e-10,0.000000
7,0.000000,0.000000,20.900000,16.976495,0.848825,0.042441,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,20.900000,12.166247,0.608312,0.030416,0.000000,0.000000e+00,0.000000e+00,0.000000,11.336436,4.952588e-01,9.632600e-02,4.816300e-03,0.000000


In [491]:
df_predict_ch_p_vals = df_predict_ch_p_vals.T

df_predict_charger_power = pd.DataFrame()

for charger in df_predict_ch_p_vals.columns:
    temp_df = pd.DataFrame(df_predict_ch_p_vals[charger])
    temp_df.columns = ['Power']
    temp_df['Charger ID'] = str(charger)
    df_predict_charger_power = pd.concat([df_predict_charger_power, temp_df])

df_predict_charger_power

,Power,Charger ID
0,20.900000,0
1,3.991304,0
2,0.199565,0
3,0.009978,0
4,0.000499,0
...,...,...
19,11.336436,7
20,0.495259,7
21,0.096326,7
22,0.004816,7


In [492]:
fig = df_predict_charger_power.plot(kind="area", markers=True, title="Charger power values", color="Charger ID",
                                    template="plotly_white", y="Power", facet_row="Charger ID", facet_row_spacing=0.05,
                                    labels=dict(index="Timestep (Hour) in a Day", value="", variable=""))
fig.update_yaxes(tickformat="%b", title="", matches="y")
fig.update_xaxes(dtick="M1", tickformat="%b", matches='x', griddash="dot", tickvals=[i for i in range(0, 25, 1)])
fig.update_traces(marker_size=7)
fig.for_each_annotation(lambda a: a.update(text="", font_size=24, y=1.07))
fig.add_annotation(x=-0.08, y=0.5, text="Power (kW)", textangle=-90, 
                   xref="paper", yref="paper", font_size=14, showarrow=False)
fig.update_layout(height=800, width=850, title=dict(xanchor="center", x=0.5, font_size=24),
                  legend=dict(orientation="h", yanchor="top", y=1.05, xanchor="center", x=0.5))
fig.show()

In [493]:
df_predict_ev_penalties.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0.0,0.010761,0.0,0.422263,0.252389,0.0,0.0,0.048164,0.0,0.0,0.185198,0.01208,0.33886,0.085956,0.0,0.0,0.0,0.0,0.0,1.038598,0.096498,0.0,0.0,0.0


In [494]:
fig = df_predict_ev_penalties.plot(kind="bar", title="Vehicle penalties", template="plotly_white", 
                       labels=dict(index="Timestep (Hour) in a Day", value="Penalty value"))
fig.update_yaxes(dtick="", tickformat="%b")
fig.update_xaxes(dtick="M1", tickformat="%b", griddash="dot")
fig.update_layout(width=950, title=dict(xanchor="center", x=0.5, font_size=24, y=0.9), showlegend=False)
fig.show()

In [495]:
df_predict_bess_action.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,-0.034579,-0.2472,-0.768751,-0.988899,-0.535649,-0.823772,-1.0,-1.0,-0.639294,-0.466862,-1.0,-0.11169,-0.760056,-0.578098,-0.523236,0.178176,-0.070824,0.022888,-0.469813,-0.346228,-0.673459,-1.0,-1.0,-1.0


In [496]:
fig = df_predict_bess_action.plot(kind="bar", title="Battery actions", template="plotly_white", 
                       labels=dict(index="Timestep (Hour) in a Day", value="Action value"))
fig.update_yaxes(tickformat="%b")
fig.update_xaxes(dtick="M1", tickformat="%b", griddash="dot")
fig.update_layout(width=950, title=dict(xanchor="center", x=0.5, font_size=24, y=0.9), showlegend=False)
fig.show()

In [497]:
df_predict_bess_p_val.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-26.72249,0.0,0.0,0.0,0.0,-24.164489,0.0,0.69999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [498]:
fig = df_predict_bess_p_val.plot(title="Battery predicted power for dis/charging", line_shape= "linear", template="plotly_white", 
                       labels=dict(index="Timestep (Hour) in a Day", value="Power value (kW)"))
fig.update_yaxes(dtick="", tickformat="%b")
fig.update_xaxes(dtick="M1", tickformat="%b", griddash="dot")
fig.update_traces(marker_size=10)
fig.update_layout(width=950, title=dict(xanchor="center", x=0.5, font_size=24, y=0.9), showlegend=False)
fig.show()

In [499]:
df_predict_bess_penalties.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0.99406,0.99406,0.99406,0.99406,0.99406,0.99406,0.99406,0.99406,0.639294,0.639294,0.639294,0.639294,0.639294,0.578098,0.578098,0.0,0.0,0.022888,0.0,0.0,0.0,0.0,0.0,0.0


In [500]:
bess_soc = df_predict_battery_soc.copy()
bess_soc.columns = ['State of Charge']

bess_penalty = df_predict_bess_penalties.copy()
bess_penalty.columns = ['Penalty']

subplot_fig = make_subplots(specs=[[{"secondary_y": True}]])

fig1 = bess_soc.plot(kind="line")
fig2 = bess_penalty.plot(kind="bar")
fig2.update_traces(yaxis="y2")

subplot_fig.add_traces(fig1.data + fig2.data)
subplot_fig.update_layout(title="Predicted Battery State of Charge and Battery Penalties",
                          yaxis=dict(title="State of Charge"), yaxis2=dict(title="Penalty value"),
                          template="plotly_white")
subplot_fig.update_xaxes(dtick="M1", tickformat="%b", griddash="dot", title="Timestep (Hour) in a Day")
subplot_fig.update_layout(width=950, title=dict(xanchor="center", x=0.5, font_size=24, y=0.9), 
                          yaxis=dict(tickformat=" ,~%", layer="above traces"), 
                          yaxis2=dict(tickformat=" ,~g", layer="below traces"), showlegend=True)

def colorise(trace):
    if trace.type == "bar":
        trace.update(marker=dict(color="orange"))
    else:
        trace.update(line=dict(color="red"))

subplot_fig.for_each_trace(lambda t: colorise(t))

subplot_fig.show()

In [501]:
fig = df_predict_bess_penalties.plot(kind="bar", title="Battery penalties", template="plotly_white", 
                       labels=dict(index="Timestep (Hour) in a Day", value="Penalty value"))
fig.update_yaxes(tickformat="%b")
fig.update_xaxes(dtick="M1", tickformat="%b", griddash="dot")
fig.update_layout(width=950, title=dict(xanchor="center", x=0.5, font_size=24, y=0.9), showlegend=False)

fig.show()